In [12]:
import pandas as pd
import numpy as np

In [11]:
df = pd.read_csv('/home/djden/Development/Science/FateOfFat/MDSS/Processed Data/db_nl_iterative.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df

,возраст,пол,масса_тела,рост,от,об,оп,жм,мм,ож,...,селен_se,фтор_f,хром_cr,цинк_zn,профессия_работники_преимущественно_умственного_труда,профессия_работники_занятые_легким_физическим_трудом,спорт_легкий_спорт,спорт_не_занимаюсь,bmi,target
0,-1.280743,0.889277,-0.112726,0.039956,-0.529443,-0.166750,-0.466780,-0.052147,0.073722,-0.378190,...,-1.439910,-0.427844,-0.446195,-1.268925,-0.990685,-0.292237,-0.481511,-0.723903,-0.089426,0
1,-1.280743,-1.124509,-0.038041,0.256187,-0.440526,-0.312860,-0.340663,-0.155567,0.266659,-0.273770,...,-1.439471,-0.345694,-0.460462,-1.271370,-0.990685,-0.292237,-0.481511,-0.723903,-0.130846,0
2,-1.167063,0.889277,-0.125471,0.124314,-0.509102,-0.136125,-0.511137,-0.080583,0.049157,-0.365340,...,-1.438300,-0.339956,-0.428594,-1.257924,1.009403,-0.292237,-0.481511,-0.723903,-0.114785,0
3,-1.110223,-1.124509,-0.036409,0.155958,-0.367017,0.006600,-0.166171,-0.120612,0.247901,-0.154877,...,-1.440495,-1.207906,-0.322849,-1.286038,-0.990685,-0.292237,-0.481511,-0.723903,-0.042369,0
4,-1.167063,-1.124509,-0.012481,0.098590,-0.270574,0.102380,0.047759,-0.194747,0.295993,-0.172524,...,-1.439178,-1.271361,-0.488772,-1.279926,-0.990685,-0.292237,-0.481511,-0.723903,-0.024170,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2880,0.083417,-1.124509,0.646373,0.218330,0.392358,0.449738,-0.082955,0.212807,0.571591,1.242420,...,-0.562582,-0.390429,-0.538136,-0.729868,1.009403,-0.292237,-0.481511,1.381400,0.576100,0
2881,0.992857,-1.124509,3.715288,0.651990,3.832604,2.760444,0.729550,3.628583,1.442523,2.801017,...,2.276952,0.702250,-0.511315,1.650049,1.009403,-0.292237,-0.481511,1.381400,3.343915,1
2882,0.253937,0.889277,-0.239666,-0.648990,-0.153596,-0.440510,-0.734538,0.093146,-1.227878,-0.757549,...,0.699840,0.744819,1.623603,0.271237,1.009403,-0.292237,-0.481511,1.381400,0.018498,1
2883,0.140257,-1.124509,3.497806,1.844554,2.523515,1.233891,6.687559,2.317751,2.607389,4.704436,...,1.478955,-0.216596,-0.465337,0.893414,1.009403,-0.292237,-0.481511,1.381400,2.350623,1


---
K-Fold Cross-Validation
---

In [13]:
X = df.drop(columns=['target'])
y = df['target']

In [18]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

pipeline = Pipeline([
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(n_estimators=300, max_depth=10, min_samples_leaf=2, min_samples_split=5, random_state=42))
])

scores = cross_val_score(pipeline, X, y, cv=kfold, scoring='accuracy')

In [19]:
scores

array([0.7729636 , 0.79722704, 0.77989601, 0.81975737, 0.81975737])

In [20]:
from sklearn.feature_selection import RFE

classifier = RandomForestClassifier(n_estimators=300, random_state=42, max_depth=10, min_samples_leaf=2, min_samples_split=5)

rfe = RFE(estimator=classifier, n_features_to_select=10, step=1)
rfe.fit(X, y)

RFE(estimator=RandomForestClassifier(max_depth=10, min_samples_leaf=2,
                                     min_samples_split=5, n_estimators=300,
                                     random_state=42),
    n_features_to_select=10)

In [21]:
X.columns[rfe.support_]

Index(['возраст', 'масса_тела', 'рост', 'от', 'об', 'жм', 'тощая_мт', 'акм',
       'гамма-линолен._к-та', 'bmi'],
      dtype='object')

Далее нужно сделать понижение размерности (от исходного датасета) при помощи PCA, t-SNE, UMAP.

Потом кластеризацию при помощи FAISS, HNSW

Возможно убрать RFE, добавить Hyperparameter tuning using GridSearchCV